In [ ]:
# BiLSTM

#from random import random
#from numpy import array
#from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
#from keras.layers import Bidirectional
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense

filename = 'asdf'

inputs = prep_features(filename)
tempos = blah


# inputs and outputs
nfft = 
nwin = 
a1 = Input(shape=(nwin,nfft/2 + 1))
a2 = Input(shape=(nwin,nfft/2 + 1))
a3 = Input(shape=(nwin,nfft/2 + 1))
a4 = Input(shape=(nwin,nfft/2 + 1))
a5 = Input(shape=(nwin,nfft/2 + 1))
a6 = Input(shape=(nwin,nfft/2 + 1))
alist = [a1,a2,a3,a4,a5,a6]
a = keras.layers.Concatenate(axis=-1)(alist)

def fork (model, n=2):
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
    return forks

# fork the inputs
left, right = fork(a)

# LSTM left branch
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
# LSTM right branch
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))

# combine
output = Sequential()
output.add(Merge([left, right], mode='concat'))
# add dense at end - beat or no beat are the 2 units at end
output.add(TimeDistributed(Dense(units=2, activation='softmax')))

# stochastic gradient descent vs ADAM
adm = Adam(lr=0.0001, decay=1e-5, momentum=0.9, nesterov=True)

# create computational graph 
model = Model(inputs=alist, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=adm)

print("Train...")
model.fit(X_train, Y_train, batch_size=1, nb_epoch=nb_epoches, validation_data=(X_test, Y_test), verbose=1, show_accuracy=True)




In [ ]:
# 6 spectrograms

import librosa
from scipy.signal import medfilt

def prep_features(x, fs):
    
    #x, fs = librosa.load(filename, sr=44100)

    # 3 melspectrograms
    s1 = librosa.core.stft(x, n_fft=1024, hop_length=441, win_length=1024, window='hamming', center=True, pad_mode='constant')
    ms1 = librosa.feature.melspectrogram(sr=44100, S=s1)
    s2 = librosa.core.stft(x, n_fft=2048, hop_length=441, win_length=2048, window='hamming', center=True, pad_mode='constant')
    ms2 = librosa.feature.melspectrogram(sr=44100, S=s2)
    s3 = librosa.core.stft(x, n_fft=4096, hop_length=441, win_length=4096, window='hamming', center=True, pad_mode='constant')
    ms3 = librosa.feature.melspectrogram(sr=44100, S=s3)

    # 3 median spectrograms
    # pad edges
    p1 = numpy.pad(ms1, ((0,0),(0,1024//2)), 'reflect')
    p2 = numpy.pad(ms2, ((0,0),(0,2048//2)), 'reflect')
    p3 = numpy.pad(ms3, ((0,0),(0,4096//2)), 'reflect')
    # cut off the non-causal median leftover from the end
    m1 = medfilt(p1, kernel_size=(1,1024//100))[:-1024/2]
    m2 = medfilt(p2, kernel_size=(1,2048//100))[:-2048/2]
    m3 = medfilt(p3, kernel_size=(1,4096//100))[:-4096/2]
    # half-wave rectify
    h1 = np.maximum(ms1-m1,0)
    h2 = np.maximum(ms2-m2,0)
    h3 = np.maximum(ms3-m3,0)

    return ms1, ms2, ms3, h1, h2, h3

In [ ]:
# CNN

from keras.layers import Conv1D
from keras.layers import Flatten

def cnn(x, num_steps, win_size=1024, weight_decay=0.01):

    a = Input(shape=(num_steps, win_size))
    filters = 256
    kernel_size = 128
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(a)
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(net)
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(net)
    # remove channel dimension
    net = Flatten(data_format='channels_last')(net)
    
    net = Dense(units=128, activation='linear')

def fork (model, n=2):
    
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
        
    return forks


def bilstm(inputs):

    # fork the inputs
    left, right = fork(inputs)

    # LSTM left branch
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    # LSTM right branch
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))

    # combine
    output = Sequential()
    output.add(Merge([left, right], mode='concat'))
    # add dense at end - beat or no beat are the 2 units at end
    output.add(TimeDistributed(Dense(units=2, activation='softmax')))
    
    return output

In [15]:
from scipy.io import loadmat

def load_annotations(filename):

    x = loadmat(filename)
    beats = x['datarec'][10]
    tempo = x['datarec'][8]

    y = beats[0].flatten()
    d = {
        'beats':y,
        'tempo':tempo[0][0][0]
    }
    return d

#print(prep_data("./dataset/info/001_info.mat"))


{'beats': array([  20064,   46050,   71272,   97640,  124678,  152018,  178895,
        204642,  231774,  258143,  284129,  310879,  338012,  365144,
        390748,  416352,  443600,  468821,  497669,  524233,  550219,
        576205,  603338,  629324,  656839,  683207,  709957,  736326,
        763076,  789445,  816195,  842563,  868282,  895032,  921401,
        948151,  973373, 1000505, 1025345, 1051713, 1078846, 1106361,
       1132347, 1158715, 1185848, 1212565, 1238708, 1264363, 1291428,
       1317804, 1343790, 1370922, 1396909, 1422513, 1448881, 1474867,
       1501617, 1529132, 1555118, 1582633, 1608619, 1634987, 1660209,
       1687552, 1714971, 1740985, 1766769, 1793312, 1820827, 1847577,
       1872799, 1899549, 1926682, 1953050, 1979418, 2006169, 2032919,
       2060434, 2085656, 2111260, 2139017, 2166863, 2193785, 2219367,
       2246450, 2272848, 2298559, 2323586, 2349983, 2376380, 2402092,
       2429518, 2456258, 2483805]), 'tempo': 99.9563288322695}


In [ ]:
import os
import librosa

def prep_hainsworth_data(info_dir, sample_dir):
    audio_array = []
    label_array = []

    #infodir = "./info/"
    #sampledir = "./samples/"

    for i in range(245):
        # {} for replacement, : format spec, 03 pad up to 3 leading 0s, d is int
        filename1 = "{:03d}_info.mat".format(i)
        filename2 = "{:03d}.wav".format(i)

        f1 = os.path.join(info_dir, filename1)
        f2 = os.path.join(sample_dir, filename2)
        if os.path.exists(f1) and os.path.exists(f2):
            # TODO: hopsize and target sample rate
            x, fs = librosa.load(f2, sr=None)
            r = load_annotations(f1)
            audio_array.append(x)
            label_array.append(r) # has tempos too

    return audio_array, label_array


In [ ]:
info_dir = "./info/"
sample_dir = "./samples/"
hop_size = 1234
target_fs = 1234

# prepare the data
a, r = prep_hainsworth_data(info_dir, sample_dir, hop_size, target_fs)
#a, r = prep_ballroom_data(info_dir, sample_dir, hop_size, target_fs)

# 6 spectrograms
spectrogram_array = []
for i in a:
    spectrogram_array.append(prep_features(i, target_fs))

# create BiLSTM and dense

# batch dataset

# training

# testing
